In [1]:
import tensorflow as tf
import inception_v3
from datagenerator import ImageDataGenerator
from tensorflow.contrib.data import Iterator
import numpy as np
from datetime import datetime
import time
import os
import json
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Use same number for training and validation
# Ex) 0th folding -> 'train0' for training, 'valid0' for validation
data_txt = open('/HS_code/ICROS_data/data_split/5fold_0802.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)
print(tr_data_dir.keys())

dict_keys(['train0', 'valid1', 'train3', 'train4', 'valid2', 'valid3', 'valid0', 'train2', 'valid4', 'train1'])


In [3]:
# Path to the textfiles for the trainings and validation set
num = 3
date = "20180810"
train_file = '/HS_code/0_Code/train%d_HR_ICROS.txt'%num
val_file1 = '/HS_code/0_Code/valid%d_HR_ICROS.txt'%num
batch_size = 128
num_classes = 2
learning_rate = 0.001
training_epochs = 50
display_step = 20

# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = "/HS_code/2_Result_TB/tensorboard_HS_%s_GoogLeNet_hr%d_valid%d_%depoch_0_001"%(date,num,num,training_epochs)
checkpoint_path = "/HS_code/1_Model_CP/checkpoints_HS_%s_GoogLeNet_hr%d_valid%d_%depoch_0_001"%(date,num,num,training_epochs)

In [4]:
with tf.device('/gpu:0'):
    # data load
    tr_data = ImageDataGenerator(train_file,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=True)
    val_data1 = ImageDataGenerator(val_file1,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)

    # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                           tr_data.data.output_shapes)

    next_batch = iterator.get_next()
    

    # Ops for initializing the two different iterators
    training_init_op = iterator.make_initializer(tr_data.data)
    validation_init_op1 = iterator.make_initializer(val_data1.data)

    # TF placeholder for graph input and output
    x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
    y = tf.placeholder(tf.float32, [batch_size, num_classes])


    net, net_points = inception_v3.inception_v3(x, 
                                              num_classes=num_classes,
                                              dropout_keep_prob=0.5
                                              )
        
    # Op for calculating the loss
    with tf.name_scope("cross_ent"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
                
    # Train op
    with tf.name_scope("train"):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
            
    # Add the loss to summary
    tf.summary.scalar('cross_entropy', loss)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Add the accuracy to the summary
    tf.summary.scalar('accuracy', accuracy)
    
    # Merge all summaries together
    merged_summary = tf.summary.merge_all()

    # Initialize the FileWriter
    writer = tf.summary.FileWriter(filewriter_path)

    # Initialize an saver for store model checkpoints
    saver = tf.train.Saver()

    train_batches_per_epoch = int(np.floor(tr_data.data_size / batch_size))
    val_batches_per_epoch1 = int(np.floor(val_data1.data_size / batch_size)) 
    print(train_batches_per_epoch)
    print(val_batches_per_epoch1)

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.
28
7


In [5]:
# Training
vaild_result = np.array([])
vaild_result.resize((training_epochs,1))


config=tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=tf.get_default_graph()) as sess:

    
    save_path = os.path.join(checkpoint_path,'model_epoch'+str(50)+'.ckpt')
    saver.restore(sess, save_path)
    
    img_batch = np.zeros((batch_size,227,227,3), dtype ='uint8')
       
    # Validate the model on the entire validation set
    print("{} Start validation".format(datetime.now()))
    sess.run(validation_init_op1)
    test_acc = 0.
    test_count = 0
    wrong_cnt = 0
    pre = np.array([])
    pre.resize((val_batches_per_epoch1,batch_size))

    for step in range(val_batches_per_epoch1):

        img_batch, label_batch = sess.run(next_batch)
        acc = sess.run(accuracy, feed_dict={x: img_batch,
                                            y: label_batch})
        pre[test_count] = sess.run(correct_prediction, feed_dict={x: img_batch,
                                            y: label_batch})

        
        pre = pre.astype('uint32')
        for i in range(batch_size):
            if pre[test_count][i] == False:
                order = step*batch_size + i
                set_num = num
                temp = tr_data_dir['train%d'%set_num][order]
                name = temp.split('/')[-1].split('.')[0].split('(')[1].split(')')[0]
                print(name)

        test_acc += acc
        test_count += 1

    test_acc /= test_count
    print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                   test_acc))

  

INFO:tensorflow:Restoring parameters from /HS_code/1_Model_CP/checkpoints_HS_20180810_GoogLeNet_hr3_valid3_50epoch_0_001/model_epoch50.ckpt
2018-08-10 10:24:06.336735 Start validation
573
377
1081
1077
869
1786
974
1137
1106
855
192
1291
93
439
538
1386
1995
774
2092
1628
1960
1820
1643
1781
788
1005
1059
79
2329
1408
2279
912
1388
471
1591
1546
1788
1923
739
1958
2004
2018-08-10 10:24:11.402264 Validation Accuracy = 0.9554


# 맞추면 1, 틀리면 0
test_count = 0
config=tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
config.gpu_options.allow_growth = True
pre = np.array([])
pre.resize((val_batches_per_epoch1,batch_size))
with tf.Session(config=config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer())
    img_batch = np.zeros((batch_size,227,227,3), dtype ='uint8')
    for epoch in range(training_epochs):
        # Initialize iterator with the training dataset
        sess.run(validation_init_op1)
        for step in range(val_batches_per_epoch1):
            # get next batch of data
            img_batch, label_batch = sess.run(next_batch)    
            #print(len(label_batch))
            pre[test_count] = sess.run(correct_prediction, feed_dict={x: img_batch,
                                                y: label_batch})
            pre = pre.astype('uint32')
            for i in range(batch_size):
                if pre[test_count][i] == False:
                    order = step*batch_size + i
                    set_num = num+1
                    temp = tr_data_dir['train%d'%set_num][order]
                    name = temp.split('/')[-1].split('.')[0].split('(')[1].split(')')[0]
                    print(name)
            test_count+=1
            
